In [1]:

import hashlib
import os
import random
import time
from datetime import datetime
from io import BytesIO

import pandas as pd
import praw
import requests
import torch
from PIL import Image
from dask.diagnostics import ProgressBar
from pmaw import PushshiftAPI
from tqdm import tqdm

from shared_code.utility.schemas.pyarrow_schema import schema
from shared_code.utility.spark.set_environ import set_azure_env

set_azure_env()

from shared_code.utility.storage.azure_file_storage import AzureFileStorageAdapter
from shared_code.utility.scripts.blip_caption import BlipCaption

pbar = ProgressBar()
pbar.register()

tqdm.pandas()
tqdm.pandas(desc="global")

from tqdm.dask import TqdmCallback

cb = TqdmCallback(desc="global")
cb.register()

file_system = AzureFileStorageAdapter('data').get_file_storage()

In [2]:
%%time

subs = [
	"SFWRedheads",
	"sfwpetite",
	"SFWNextDoorGirls",
	"trippinthroughtime",
	"fatsquirrelhate",
	"itookapicture",
	"memes",
	"WhitePeopleTwitter",
	"CityPorn",
	"EarthPorn",
	"spaceporn",
	"realasians",
	"KoreanHotties",
	"prettyasiangirls",
	"AsianOfficeLady",
	"mildlypenis",
	"cougars_and_milfs_sfw",
	"AsianInvasion",
	"sexygirls",
	"PrettyGirls",
	"gentlemanboners",
	"hotofficegirls",
	"tightdresses",
	"DLAH"
]


CPU times: total: 0 ns
Wall time: 0 ns


In [3]:
%%time

extant_data = pd.read_parquet("data/processed_raw_data.parquet", engine='pyarrow', filesystem=file_system, schema=schema)
display(extant_data)

,id,subreddit,author,title,caption,hash,permalink,original_url,image_name,path,model,exists,curated,accept,tags
0,128z5ns,celebrities,A-Sexy-Name,Rachel Weisz,cars are parked on the side of the road in the...,f09b1c0c5dea50c934081a04c83d4d33,/r/celebrities/comments/128z5ns/rachel_weisz/,https://i.redd.it/nv80afddacra1.jpg,f09b1c0c5dea50c934081a04c83d4d33.jpg,D:\data\images\celebrities\f09b1c0c5dea50c9340...,None,True,False,None,None
1,11yk8db,prettyasiangirls,MisoShiru520,Library,blonde woman with blonde hair and tattoos on h...,9d5b7331b941ecf3fdbc8ad322303825,/r/prettyasiangirls/comments/11yk8db/library/,https://i.redd.it/ht4d2wuusapa1.jpg,9d5b7331b941ecf3fdbc8ad322303825.jpg,D:\data\images\prettyasiangirls\9d5b7331b941ec...,None,True,False,None,None
2,11hfioi,KoreanHotties,Majestic_Painter8660,Underboob bikini,a man with a beard and a beard sitting in fron...,a154a56b1b87cf3a0bc93600c86e4585,/r/KoreanHotties/comments/11hfioi/underboob_bi...,https://i.redd.it/cimr17q84jla1.jpg,a154a56b1b87cf3a0bc93600c86e4585.jpg,D:\ImageDoesNotExist.png,None,False,False,None,None
3,1000mjs,spaceporn,MorningStar_imangi,Northern Lights above Lofoten,a view of a view of a large green and purple a...,2c39ce1290fba541abd0b004b09da6b2,/r/spaceporn/comments/1000mjs/northern_lights_...,https://i.redd.it/7s5aafaqkb9a1.jpg,7s5aafaqkb9a1.jpg,D:\data\images\spaceporn\7s5aafaqkb9a1.jpg,None,True,False,None,None
4,1000qpd,spaceporn,MorningStar_imangi,Viking Lights,a scene of a boat is sitting on the shore of a...,0f72de47c69ff50eca5fa3990215f4ac,/r/spaceporn/comments/1000qpd/viking_lights/,https://i.redd.it/abojw7lqlb9a1.jpg,abojw7lqlb9a1.jpg,D:\data\images\spaceporn\abojw7lqlb9a1.jpg,None,True,False,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41500,131dso7,DLAH,Wallydinger123,Happy B Day,there are three women posing for a picture in ...,ebb4b4a88dc79e2f2f57b10f32bf03c0,/r/DLAH/comments/131dso7/happy_b_day/,https://i.redd.it/36gurz533jwa1.jpg,ebb4b4a88dc79e2f2f57b10f32bf03c0.jpg,D:\data\images\DLAH\ebb4b4a88dc79e2f2f57b10f32...,None,True,False,None,None
41501,131eznp,DLAH,Long_dong111,Perfect,araffe woman in a white dress sitting on a pur...,5c944c6d62b1ae182c5a6a40c64195f4,/r/DLAH/comments/131eznp/perfect/,https://i.redd.it/dnbbx4zucjwa1.jpg,5c944c6d62b1ae182c5a6a40c64195f4.jpg,D:\data\images\DLAH\5c944c6d62b1ae182c5a6a40c6...,None,True,False,None,None
41502,131g3am,DLAH,ThrownAwayMiles,DLAH left or right,two women in short dresses posing for a pictur...,ab1f8fc33c85f4cfce12c810cf22229e,/r/DLAH/comments/131g3am/dlah_left_or_right/,https://i.redd.it/zpp70oor3lwa1.jpg,ab1f8fc33c85f4cfce12c810cf22229e.jpg,D:\data\images\DLAH\ab1f8fc33c85f4cfce12c810cf...,None,True,False,None,None
41503,131jvcx,DLAH,robok212,Good morning 😍,araffe in a pink dress posing on a white wall,01f5058855526ef25888cff88065db4c,/r/DLAH/comments/131jvcx/good_morning/,https://i.redd.it/ozmbfhjv3mwa1.jpg,01f5058855526ef25888cff88065db4c.jpg,D:\data\images\DLAH\01f5058855526ef25888cff880...,None,True,False,None,None


CPU times: total: 344 ms
Wall time: 1.34 s


In [4]:
%%time

reddit: praw.Reddit = praw.Reddit(client_id='5hVavL0PIRyM_1JSvqT6UQ', client_secret='BjD2kS3WNLnJc59RKY-JJUuc_Z9-JA', user_agent='script:%(bot_name)s:v%(bot_version)s (by /u/%(bot_author)s)')
api: PushshiftAPI = PushshiftAPI(praw=reddit, num_workers=12)

CPU times: total: 31.2 ms
Wall time: 139 ms


Version 7.6.1 of praw is outdated. Version 7.7.0 was released Saturday February 25, 2023.


In [5]:
%%time

from dask.callbacks import Callback
from tqdm.autonotebook import tqdm


class TQDMDaskProgressBar(Callback, object):
	"""
    A tqdm progress bar for dask.
    Usage:
        ```
        with TQDMDaskProgressBar():
            da.compute()
        ```
    Author : wassname
    Source : https://gist.github.com/wassname/1837d0365247430e02abda41f0e7f184
    See: http://dask.pydata.org/en/latest/diagnostics-local.html?highlight=progress
    """

	def __init__(self, start=None, start_state=None, pretask=None, posttask=None, finish=None, **kwargs):
		super(TQDMDaskProgressBar, self).__init__(
			start=start,
			start_state=start_state,
			pretask=pretask,
			posttask=posttask,
			finish=finish,
		)
		self.tqdm_args = kwargs
		self.states = ["ready", "waiting", "running", "finished"]

	def _start_state(self, dsk, state):
		self._tqdm = tqdm(total=sum(len(state[k]) for k in self.states), **self.tqdm_args)

	def _posttask(self, key, result, dsk, state, worker_id):
		self._tqdm.update(1)

	def _finish(self, dsk, state, errored):
		self._tqdm.close()

CPU times: total: 0 ns
Wall time: 0 ns


In [6]:
def filter_by_image_post(item) -> bool:
	return not (item['selftext'].__contains__('[removed]') or item['selftext'].__contains__('[deleted]') and not item[
		'url'].endswith('.jpg'))

In [7]:
def fetch_image(x: object) -> object:
	try:
		time.sleep(0.1)
		url = x['original_url']
		subreddit = x['subreddit']
		response = requests.get(url)
		md5 = hashlib.md5(response.content).hexdigest()
		os.makedirs(f"D:\\data\\images\\{subreddit}", exist_ok=True)
		out_path = f"D:\\data\\images\\{subreddit}\\{md5}.jpg"

		if os.path.exists(out_path):
			return out_path

		if md5 in list(extant_data['hash']):
			return out_path

		if md5 != "f17b01901c752c1bb04928131d1661af" or md5 != "d835884373f4d6c8f24742ceabe74946":
			raw_image = Image.open(BytesIO(response.content))
			raw_image.save(out_path)
			raw_image.close()
			return out_path
		else:
			return ""
	except Exception as e:
		print(e)
		return ""

In [8]:
def exists(x: object) -> bool:
	try:
		image_path = os.path.exists(x['path'])
		return image_path
	except Exception as e:
		print(e)
		return False

In [9]:
def split_hash(x: object) -> str:
	try:
		name = x['image_name']
		return name.split('.')[0]
	except Exception as e:
		print(e)
		return ""

In [10]:
def caption_image(x: object, blip_captioning_list: [BlipCaption, BlipCaption]) -> str:
	try:
		path = x['path']
		exists_image = bool(x['exists'])
		resulting_caption = x['caption']

		if not os.path.exists(path):
			return ""

		if not exists_image:
			return ""

		if resulting_caption == None:
			pass
		if resulting_caption is None:
			pass
		else:
			if len(resulting_caption) > 6:
				return resulting_caption
			if resulting_caption != "":
				return resulting_caption
			else:
				pass

		blip_captioning = random.choice(blip_captioning_list)

		resulting_caption = blip_captioning.caption_image(path)

		return resulting_caption

	except Exception as e:
		print(e)
		return ""
	finally:
		pass

In [11]:
def get_image_name(x: object) -> str:
	try:
		path = x['path']
		if path == "":
			return ""
		return os.path.basename(path)
	except Exception as e:
		print(e)
		return ""

In [12]:
from praw.models import ListingGenerator
import pandas as pd

posts = []
for sub in tqdm(subs, total=len(subs), desc="Creating subs"):
	if not os.path.exists(f"{sub}"):
		os.makedirs(f"{sub}")
	existing_data = pd.read_parquet(f"{sub}")
	os.makedirs(f"{sub}", exist_ok=True)

Creating subs:   0%|          | 0/24 [00:00<?, ?it/s]

In [ ]:
for time_filter in ['all', 'year', 'month', 'week', 'day']:
	for sub in tqdm(subs, desc=f"{time_filter}", total=len(subs)):
		existing_data = pd.read_parquet(f"{sub}")
		try:
			subreddit_stream: ListingGenerator = reddit.subreddit(display_name=sub).top(limit=100, time_filter=time_filter)
			subreddit_stream = list(subreddit_stream)
			for submission in tqdm(subreddit_stream, total=len(subreddit_stream), desc=f"Posts - {sub} - {time_filter}"):
				if submission is None:
					continue
				if not os.path.exists(f"{sub}"):
					os.makedirs(f"{sub}")
				else:
					if submission.id in list(extant_data['id']):
						continue
				try:
					author_name = 'Unknown'
					subreddit_name = sub
					try:
						author_name = submission.author.name
					except Exception as e:
						author_name = 'Unknown'
						pass
					p = {
						'id': submission.id,
						'subreddit': subreddit_name,
						'author': author_name,
						'title': submission.title,
						'caption': '',
						'hash': '',
						'permalink': submission.permalink,
						'original_url': submission.url,
						'image_name': '',
						'path': '',
						'thumbnail_path': '',
						'exists': False,
						'curated': False
					}
					pd.DataFrame([p]).to_parquet(f"{sub}/{submission.id}-{time_filter}.parquet")
				except Exception as e:
					# print(e)
					continue
		except Exception as e:
			print(e)
			continue

all:   0%|          | 0/24 [00:00<?, ?it/s]

Posts - SFWRedheads - all:   0%|          | 0/100 [00:00<?, ?it/s]

Posts - sfwpetite - all:   0%|          | 0/100 [00:00<?, ?it/s]

Posts - SFWNextDoorGirls - all:   0%|          | 0/100 [00:00<?, ?it/s]

Posts - trippinthroughtime - all:   0%|          | 0/100 [00:00<?, ?it/s]

Posts - fatsquirrelhate - all:   0%|          | 0/100 [00:00<?, ?it/s]

Posts - itookapicture - all:   0%|          | 0/100 [00:00<?, ?it/s]

Posts - memes - all:   0%|          | 0/100 [00:00<?, ?it/s]

In [ ]:
%%time
dfs = pd.DataFrame({}, columns=schema.names)
data = []
for sub in subs:
	df = pd.read_parquet(f"{sub}")
	initial = pd.DataFrame({}, columns=schema.names)
	initial.id = df.id
	initial.subreddit = df.subreddit
	initial.author = df.author
	initial.title = df.title
	initial.caption = ""
	initial.hash = ""
	initial.permalink = df.permalink
	initial.original_url = df.original_url
	initial.image_name = ""
	initial.path = ""
	initial.thumbnail_path = ""
	initial.exists = False
	initial.curated = False
	initial.accept = False
	initial.tags = []

	records = df.to_dict(orient='records')

	data.extend(records)

initial = pd.DataFrame(data=data, columns=schema.names)
filtered_initial = initial.where(initial['original_url'].str.endswith('.jpg')).dropna(how='all').reset_index().drop('index', axis=1)

display(filtered_initial)

In [ ]:
%%time

tqdm.pandas(desc="Filtering Exists For Id List")
with ProgressBar():
	extant = extant_data.where((extant_data['exists'].isin([True]))).dropna(how='all').reset_index().drop('index', axis=1)

extant_id_list = list(extant.id)

tqdm.pandas(desc="Filtering Based on Id")
with ProgressBar():
	new_entries = filtered_initial.where((~filtered_initial['id'].isin(extant_id_list))).dropna(
		how='all').reset_index().drop('index', axis=1)

display(new_entries)

new_images_by_sub = new_entries.groupby('subreddit')

new_images_by_sub.count().plot.bar(y='id')

In [ ]:
%%time

new_entries_1 = new_entries.copy()

tqdm.pandas(desc="Fetch-Image")
with ProgressBar():
	new_entries_1['path'] = new_entries_1.progress_apply(lambda x: fetch_image(x), axis=1)

tqdm.pandas(desc="Get-Image-Name")
with ProgressBar():
	new_entries_1['image_name'] = new_entries_1.progress_apply(lambda x: get_image_name(x), axis=1)

tqdm.pandas(desc="Split-Hash")
with ProgressBar():
	new_entries_1['hash'] = new_entries_1.progress_apply(lambda x: split_hash(x), axis=1)

tqdm.pandas(desc="Checking-Exists")
with ProgressBar():
	new_entries_1['exists'] = new_entries_1.progress_apply(lambda x: exists(x), axis=1)

display(new_entries_1)

In [ ]:
%%time

blip_0 = BlipCaption(1)
blip_1 = BlipCaption(0)

In [ ]:
%%time

torch.cuda.empty_cache()
new_entries_final = new_entries_1.copy()

tqdm.pandas(desc="Caption-Image")
with ProgressBar():
	new_entries_final['caption'] = new_entries_1.progress_apply(lambda x: caption_image(x, [blip_0, blip_1]), axis=1)
display(new_entries_final)

In [ ]:
# extant_data.set_index('id', inplace=True)
# extant_data.update(new_entries_final.set_index('id'))
# extant_data.reset_index()
#
# display(extant_data)

In [ ]:
%%time

tqdm.pandas(desc="Filtering")
writeable_entries = new_entries_final \
	.where(~new_entries_final['id'].isin(extant_data['id'])) \
	.dropna(how='all') \
	.reset_index() \
	.drop('index', axis=1)

display(writeable_entries)

In [ ]:
%%time

combined_result = pd.concat([extant_data, writeable_entries])

display(combined_result)

In [ ]:
%%time

combined_result.reindex()
combined_result.to_csv('processed_raw_data.csv', index=False)
f = AzureFileStorageAdapter('data').get_file_storage()
f.put('processed_raw_data.csv', 'data/processed_raw_data.csv', overwrite=True)

In [ ]:
%%time

final = pd.read_csv(f.open('data/processed_raw_data.csv'))
final.to_parquet("data/processed_raw_data.parquet", engine='pyarrow', filesystem=file_system, schema=schema)

In [ ]:
%%time

final = pd.read_parquet("data/processed_raw_data.parquet", engine='pyarrow', filesystem=file_system)
display(final)